In [1]:
import pandas as pd
import scipy
import sklearn
import numpy as np
import pickle
import setuptools
import wheel
import matplotlib.pyplot as plt
import docplex.mp.model as cpx
import random

import pulp as plp
from pulp import LpProblem, LpInteger, LpVariable, LpConstraint, LpConstraintGE, LpConstraintLE, LpConstraintEQ, LpMaximize, lpSum

In [2]:
# Parameters

RANK = range(1,11)
SCORE = [6,3,2,1,0,0,0,0,0,0]

ROOMS = {
    'RoomA': [7,16,4,8],
    'RoomB': [9,10,14,1,3],
    'RoomC': [12,17,13,2,18],
    'RoomD': [6,11,5,15]
}

In [3]:
def gen_room(room_dict):
    """ Append the word "Pair" to the pair number
    """
    
    output_dict = {}
    for key in room_dict.keys():
        templist = [''.join(['pair ', str(room)]) for room in room_dict[key]]
        output_dict[key] = templist
    return output_dict

def sum_room(old, room_dict):
    """ Sum scores of all rooms of each candidate
    """
    data = old.copy()
    #for room in room_dict.keys():
    #    data[room] = data[room_dict[room]].sum(axis = 1)
    data['RoomA'] = old['pair 4'] + old['pair 7'] + old['pair 8'] + old['pair 16']
    data['RoomB'] = old['pair 1'] + old['pair 3'] + old['pair 9'] + old['pair 10'] + old['pair 14']
    data['RoomC'] = old['pair 12'] + old['pair 17'] + old['pair 13'] + old['pair 2'] + old['pair 18']
    data['RoomD'] = old['pair 5'] + old['pair 6'] + old['pair 11'] + old['pair 15']
    return data

In [4]:
df = pd.read_csv('pair_selection.csv').fillna(0)

In [5]:
df.head()

,ID,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18
0,6230124521,3.0,2.0,0.0,0.0,4.0,10.0,0.0,9.0,0.0,7.0,0.0,8.0,0.0,0.0,5.0,0.0,1.0,6.0
1,6030274221,0.0,6.0,1.0,0.0,7.0,9.0,0.0,5.0,0.0,10.0,0.0,3.0,0.0,0.0,2.0,0.0,8.0,4.0
2,6031779021,4.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,0.0,3.0,0.0,0.0,7.0,8.0,10.0,9.0
3,6030650521,1.0,0.0,9.0,0.0,4.0,10.0,8.0,7.0,3.0,6.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0
4,6131791621,4.0,0.0,2.0,9.0,0.0,0.0,0.0,1.0,8.0,0.0,0.0,10.0,0.0,6.0,5.0,0.0,3.0,7.0


In [6]:
# Transform rank to score
transform_dict = {k: v for k, v in zip(RANK, SCORE)}
df_score = df.replace(transform_dict)
df_score.head()

,ID,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18
0,6230124521,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
1,6030274221,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0
2,6031779021,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6030650521,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,6131791621,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [7]:
new = sum_room(df_score, gen_room(ROOMS))
room_score = new[['ID', 'RoomA', 'RoomB', 'RoomC', 'RoomD']]
#room_score_unranked = new[['ID', 'RoomA', 'RoomB', 'RoomC', 'RoomD']]
room_score.head()

,ID,RoomA,RoomB,RoomC,RoomD
0,6230124521,0.0,2.0,9.0,1.0
1,6030274221,0.0,6.0,3.0,3.0
2,6031779021,6.0,4.0,2.0,0.0
3,6030650521,0.0,11.0,0.0,1.0
4,6131791621,6.0,4.0,2.0,0.0


# Define Variables for Optimization

In [8]:
I = range(0, df.shape[0]) # candidate index
J = range(1, len(ROOMS.keys()) + 1) # room index

# maximum number of candidates in each room
max_num = {1: 24, 2: 32, 3: 32, 4: 24}
min_num = {1: 23, 2: 25, 3: 25, 4: 20}

## Constraints

In [9]:
# Note: i == candidate's index, j == room's index
rank = {(i, j): room_score.iloc[i, j] for i in I for j in J}
lower_b = {(i, j): 0 for i in I for j in J}
upper_b = {(i, j): 1 for i in I for j in J}
a = upper_b.copy()

# everyone can join only one room
one_c = {(i): 1 for i in I}

# satisfaction score (need to be updated due to our level)
# (direct / grey plus / grey)
sat_c = {(i): 4 for i in I[:41]}
sat_c.update({(i): 3 for i in I[41:41 + 29]})
sat_c.update({(i): 2 for i in I[41 + 29:41 + 29 + 27]})
sat_c.update({(i): 1 for i in I[41 + 29 + 27:]})

In [10]:
#rank

## Model Setup

In [11]:
model = LpProblem(name = "score_optimizer")

# Assign lower bound and upper bound to DVs
x_vars = {
    (i, j): LpVariable(cat = LpInteger, lowBound = lower_b[i, j],
                       upBound = upper_b[i, j],
                       name = 'x_{0}_{1}'.format(i, j))
    for i in I for j in J
}

# Room minimum and maximum number constraint
cts = {j: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for i in I), sense = LpConstraintLE, rhs = max_num[j],
                                           name = 'max_number_room_{0}'.format(j))) for j in J}
cts = {j: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for i in I), sense = LpConstraintGE, rhs = min_num[j],
                                           name = 'min_number_room_{0}'.format(j))) for j in J}

# A candidate must join only one room
cts = {i: model.addConstraint(LpConstraint(e = lpSum(a[i, j] * x_vars[i, j]
                                                     for j in J), sense = LpConstraintEQ, rhs = one_c[i],
                                           name = 'one_room_candidate_{0}'.format(i))) for i in I}

# Minimum Score
cts = {i: model.addConstraint(LpConstraint(e = lpSum(rank[i, j] * x_vars[i, j] 
                                                     for j in J), sense = LpConstraintGE, rhs = sat_c[i],
                                           name = 'min_satisfaction_{0}'.format(i))) for i in I}

In [12]:
# Set objective
model.sense = LpMaximize
obj = lpSum(x_vars[i, j] * rank[i, j] for i in I for j in J)
model.setObjective(obj)

# Solve!
model.solve()

1

In [13]:
plp.LpStatus[model.status]

'Optimal'

In [14]:
# Total Satisfaction Score
summary = []
for i in I:
    for j in J:
        summary.append(plp.value(x_vars[i, j] * rank[i, j]))

## Convert the result to a DataFrame

In [15]:
opt_df = pd.DataFrame.from_dict(x_vars, orient = 'index',
                                columns = ['variable_object'])
opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, names = ['candidate_id', 'room'])
opt_df.reset_index(inplace = True)

opt_df['solution_value'] = opt_df['variable_object'].apply(lambda item: item.varValue)

final_table = opt_df.pivot(index = 'candidate_id', columns = 'room', values = 'solution_value')

In [16]:
final_table.head()

room,1,2,3,4
candidate_id,,,,
0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [17]:
temp_final = final_table.reset_index(drop = True)
temp_final['ID'] = room_score['ID']

In [18]:
temp_final.head()

room,1,2,3,4,ID
0,0.0,0.0,1.0,0.0,6230124521
1,0.0,1.0,0.0,0.0,6030274221
2,1.0,0.0,0.0,0.0,6031779021
3,0.0,1.0,0.0,0.0,6030650521
4,1.0,0.0,0.0,0.0,6131791621


In [25]:
temp_final.to_excel('final_flex_no_ranked.xlsx', index = False)

In [20]:
sum(summary)

829.0

In [21]:
room_score.head()

,ID,RoomA,RoomB,RoomC,RoomD
0,6230124521,0.0,2.0,9.0,1.0
1,6030274221,0.0,6.0,3.0,3.0
2,6031779021,6.0,4.0,2.0,0.0
3,6030650521,0.0,11.0,0.0,1.0
4,6131791621,6.0,4.0,2.0,0.0


In [22]:
df_score.head()

,ID,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18
0,6230124521,2.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
1,6030274221,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0
2,6031779021,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6030650521,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,6131791621,1.0,0.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [23]:
temp_final.sum(axis = 0).astype(int)

room
1               23
2               29
3               32
4               24
ID    658274929168
dtype: int64